# Export Subset of OpenQuake Results

Sometimes `--export-outputs` produces way too much data, and it's handy to zip up the result.

*Note that this script must be run from a notebook/jupyter environment run from a terminal in a python 2.7 environment. Switching kernels is not enough.*

In [7]:
import os
import sys
import tarfile
import subprocess

In [8]:
# configuration
CALC_ID = 123
export_dir = '../HazardOutputs'
result_dir = os.path.join(export_dir, 'calc_%d' % CALC_ID)
all_outputs_file = os.path.join(result_dir, 'all_outputs.txt')
output_keywords = ['Hazard Curves', 'Hazard Curve (multiple imts)',
                   'Hazard Map', 'UHS']
type_keywords = ['mean', 'quantile']

In [9]:
if not os.path.exists(export_dir):
    os.mkdir(export_dir)
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [10]:
# generate listing of available results
command = ['oq-engine', '--list-outputs', str(CALC_ID)]
print('Listing outputs with "%s > %s"' 
      % (' '.join(command), all_outputs_file))
sys.stdout.flush()
with open(all_outputs_file, 'w') as lho_file:
    subprocess.call(command, stdout=lho_file, stderr=subprocess.STDOUT)

Listing outputs with "oq-engine --list-outputs 123 > ../HazardOutputs/calc_123/all_outputs.txt"


In [11]:
# export results of interest
exported_files = []
n_exported, n_results = 0, 0
with open(all_outputs_file, 'r') as lho_file:
    for line in lho_file:
        try:
            result_id = int(line.split()[0])
            n_results += 1
        except ValueError:
            print line.strip()
            continue
        
        is_output = any([key.lower() in line.lower() 
                         for key in output_keywords])
        is_type = any([key.lower() in line.lower() 
                       for key in type_keywords])
        if is_output and is_type:
            print line.strip()
            sys.stdout.flush()
            command = ['oq-engine', '--export-output', 
                       str(result_id), export_dir]
            pipes = subprocess.Popen(command, 
                                     stdout=subprocess.PIPE, 
                                     stderr=subprocess.PIPE)
            std_out, std_err = pipes.communicate()
            if pipes.returncode != 0:
                err_msg = "%s\nReturn Code: %s" % (std_err.strip(), 
                                                   pipes.returncode)
                raise Exception(err_msg)
            elif len(std_err):
                print(std_err.strip())
            
            exported_file = std_out.replace('File Exported:','').strip()
            exported_files.append(os.path.relpath(exported_file))

print('Exported %d out of %d results' % (len(exported_files), n_results))

id | output_type | name
87262 | Hazard Curve | Mean Hazard Curves PGA
87263 | Hazard Curve | Mean Hazard Curves SA(0.2)
87264 | Hazard Curve | Mean Hazard Curves SA(1.0)
87261 | Hazard Curve (multiple imts) | mean-curves-multi-imt
110594 | Hazard Map | Mean Hazard map(0.00040397) PGA
110596 | Hazard Map | Mean Hazard map(0.00040397) SA(0.2)
110598 | Hazard Map | Mean Hazard map(0.00040397) SA(1.0)
110593 | Hazard Map | Mean Hazard map(0.002105) PGA
110595 | Hazard Map | Mean Hazard map(0.002105) SA(0.2)
110597 | Hazard Map | Mean Hazard map(0.002105) SA(1.0)
Exported 10 out of 38890 results


In [12]:
print('Compressing: tar -hzcvf "%s.tar.gz" "%s"' % 
      (result_dir, result_dir))
with tarfile.open(result_dir + '.tar.gz', 'w:gz') as tar:
    tar.add(result_dir, arcname=os.path.split(result_dir)[1])

Compressing: tar -hzcvf "../HazardOutputs/calc_123.tar.gz" "../HazardOutputs/calc_123"
